In [1]:
%reload_ext autoreload
%autoreload 1
%aimport train

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
tweet_df, vocab = train.load_data(num_docs=10000)

In [4]:
embedding_index_map = tweet_df[['id', 'text']]
# embedding_index_map.to_pickle('embedding_index_map.pkl')

In [5]:
docs, contexts, targets = train.prepare_contexts(tweet_df)

In [6]:
tf.reset_default_graph()

In [7]:
# model = train.build_model(num_words = len(vocab), num_docs = len(tweet_df))

# model = train.build_model(num_words = len(vocab), num_docs = len(tweet_df),
#                           doc_embedding_size=20, word_embedding_size=50)

In [8]:
# model.load('nce_test_model-10371')

In [9]:
# e1, = model.net.graph.get_collection('layer_variables/Embedding/')
# e2, = model.net.graph.get_collection('layer_variables/Embedding_1/')

# doc_embedding = model.get_weights(e1)
# word_embedding = model.get_weights(e2)

In [10]:
# train.train_model(model, docs, contexts, targets, num_words = len(vocab))

In [11]:
trainer, X1, X2, Y = train.build_nce_model(num_words = len(vocab), num_docs = len(tweet_df))

In [12]:
trainer.restore('nce_test_model-15558')

In [13]:
e1, = trainer.graph.get_collection('layer_variables/Embedding/')
e2, = trainer.graph.get_collection('layer_variables/Embedding_1/')

doc_embedding = e1.eval(session=trainer.session)
word_embedding = e2.eval(session=trainer.session)

In [14]:
vocab = np.array(vocab)

In [15]:
doc_embedding_series = pd.Series(list(doc_embedding))
embedding_index_map['embedding'] = doc_embedding_series

In [16]:
doc_embedding = np.stack(embedding_index_map.embedding.values)

In [17]:
doc_embedding_norm = doc_embedding / np.linalg.norm(doc_embedding, axis=1, keepdims=True)

In [33]:
doc = doc_embedding_norm[90]
dist_sort_idx = np.argsort(np.sum(doc * doc_embedding_norm, axis=1))[::-1]
for t in embedding_index_map.iloc[dist_sort_idx].head(20).text:
    print(t)
    print('---')

RT @AustralianLabor: "We want to make sure that every kid in Australia has a great education through their school years." @tanya_plibersek…
---
RT @AustralianLabor: "We want to make sure that every kid in Australia has a great education through their school years." @tanya_plibersek…
---
RT @JoshButler: . @GrahamPerrettMP has written us a really nice piece about tolerance and diversity in Australia https://t.co/1qPtgYFZlU
---
RT @BarackObama: Today's #SCOTUS ruling reaffirms that every woman has a right to make her own reproductive choices. https://t.co/uvgc9MgX3e
---
RT @PaulAndrews2043: Great to see this significant commitment in The Smith Family's Learning for Life. It's a program that really works. ht…
---
RT @shellzzz: Dylan Voller NT teen in custody has penned an open letter thanking Australia &amp; #FourCorners 
#Auspol Thank you Dylan https://…
---
RT @Vision2020Aus: .@Senator_McEwen has pledged to put a stop to avoidable blindness &amp; vision loss, thank you for showing South 

In [19]:
word_embedding

array([[-0.38929254,  0.01207909, -0.04467954, ..., -0.19292682,
         0.07984687, -1.03046072],
       [-0.32872912,  0.30992517, -0.24009758, ..., -0.27698398,
         0.23048559, -0.26663733],
       [-0.46030429,  0.11352346, -0.25810084, ..., -0.4582504 ,
         0.07160532, -0.62094343],
       ..., 
       [-0.30013731,  0.23166288, -0.18382449, ..., -0.19291772,
         0.20352048, -0.19774315],
       [-0.19427742,  0.18279101, -0.18957567, ..., -0.11355067,
         0.17502984, -0.15607339],
       [-0.17220894,  0.20476863, -0.31887984, ..., -0.48404208,
         0.01030864, -0.95863378]], dtype=float32)

In [20]:
word_embedding_norm = word_embedding / np.linalg.norm(word_embedding, axis=1, keepdims=True)

In [21]:
word_to_vocab = {w: i for i, w in enumerate(vocab)}

def e(s):
    return word_embedding[word_to_vocab[s]]

In [36]:
turnbull = e('@TurnbullMalcolm')
shorten = e('@billshortenmp')
liberal = e('Liberal')
labor = e('Labor')

# word = turnbull - liberal + labor
# word = shorten - labor + liberal

word = e('plebiscite') - e('gay') - e('straight')

# word = e('Liberal')
# word = e('Labor')

word_norm = word / np.linalg.norm(word)
dist_sort_idx = np.argsort(np.sum(word_norm * word_embedding_norm, axis=1))[::-1]
for t in vocab[dist_sort_idx[:20]]:
    print(t)

Billson
PortAdelaide
AusPol
Strike
closures
SpringSt
100s
loud
MyLiberal
skin
recipient
mid
cane
@eleanorbloom
@SwannyDPM
crews
parks
BER
projections
supertrawler
